In [1]:
from aiida import orm, load_profile
from aiida_aurora.calculations import BatteryCyclerExperiment

load_profile()

Profile<uuid='14f418767fa44f6f9d65ac40c9d45d39' name='default'>

In [2]:
qb = (
    orm.QueryBuilder()
    .append(
        orm.Group,
        filters={
            "label": "aurora/experiments/ga-231012-4.2",
        },
        tag="group",
    )
    .append(
        BatteryCyclerExperiment,
        with_group="group",
        tag="experiment",
    )
    .append(
        orm.ArrayData,
        with_incoming="experiment",
    )
)

In [3]:
import numpy as np

t, I, V = [], [], []  # noqa: E741
result: orm.ArrayData = qb.first(flat=True)

time = result.get_array("step0_uts")
time -= time[0]
current = result.get_array("step0_I_n")
voltage = result.get_array("step0_Ewe_n")

t = np.array(time)
I = np.array(current)
V = np.array(voltage)

In [4]:
import pandas as pd

df = pd.DataFrame({"t": t, "I": I, "Ewe": V}).apply(lambda col: col.explode())
# df

In [5]:
df.to_csv("data/test.csv", index=False)

In [6]:
df["t"] = df["t"].apply(lambda row: row / 3600)
df["I"] = df["I"].apply(lambda row: row * 1000)

In [7]:
df.to_json("data/test.json", orient="split", index=False)

# DLite

## Test 1 - simple conversion

In [8]:
# import pint

# ureg = pint.UnitRegistry()
# ureg.C

In [9]:
import dlite

In [10]:
from pathlib import Path

dlite.python_storage_plugin_path.append(Path("drivers").absolute())

In [11]:
json_instance = dlite.Instance.from_location("CyclingStorageParser", "data/test.json")

In [12]:
print(json_instance)

{
  "4b6d9e02-9b69-4df6-9178-2b8889329a59":  {
    "meta": "https://example.com/meta/0.1/CyclingTimeSeries",
    "dimensions": {
      "N": 17384
    },
    "properties": {
      "t": [0, 0.0919598, 1.31488, 8.7852, 23.715, 30, 57.0177, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900, 930, 960, 990, 1020, 1050, 1080, 1110, 1140, 1170, 1200, 1230, 1260, 1290, 1320, 1350, 1380, 1410, 1440, 1470, 1500, 1530, 1560, 1590, 1620, 1650, 1680, 1710, 1740, 1770, 1800, 1830, 1860, 1890, 1920, 1950, 1980, 2010, 2040, 2070, 2100, 2130, 2160, 2190, 2220, 2250, 2280, 2310, 2340, 2370, 2400, 2430, 2460, 2490, 2520, 2550, 2580, 2610, 2640, 2670, 2700, 2730, 2760, 2790, 2820, 2850, 2880, 2910, 2940, 2970, 3000, 3030, 3060, 3090, 3120, 3150, 3180, 3210, 3240, 3270, 3300, 3330, 3360, 3390, 3420, 3431.26, 3431.3, 3461.3, 3491.3, 3521.3, 3551.3, 3581.3, 3611.3, 3641.3, 3671.3, 3701.3, 3731.3, 3761.3, 3791.3, 3821.3

# CSV

In [13]:
cycling_time_series = dlite.Instance.from_location(
    "csv",
    "data/test.csv",
    "mode=r;meta=https://example.com/meta/0.1/CyclingTimeSeries;infer=False",
)

In [14]:
print(cycling_time_series)

{
  "226c2618-e4bc-42fd-b0e0-5988161de967":  {
    "meta": "https://example.com/meta/0.1/CyclingTimeSeries",
    "dimensions": {
      "N": 17384
    },
    "properties": {
      "t": [0, 0.09196, 1.31488, 8.7852, 23.715, 30, 57.0177, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900, 930, 960, 990, 1020, 1050, 1080, 1110, 1140, 1170, 1200, 1230, 1260, 1290, 1320, 1350, 1380, 1410, 1440, 1470, 1500, 1530, 1560, 1590, 1620, 1650, 1680, 1710, 1740, 1770, 1800, 1830, 1860, 1890, 1920, 1950, 1980, 2010, 2040, 2070, 2100, 2130, 2160, 2190, 2220, 2250, 2280, 2310, 2340, 2370, 2400, 2430, 2460, 2490, 2520, 2550, 2580, 2610, 2640, 2670, 2700, 2730, 2760, 2790, 2820, 2850, 2880, 2910, 2940, 2970, 3000, 3030, 3060, 3090, 3120, 3150, 3180, 3210, 3240, 3270, 3300, 3330, 3360, 3390, 3420, 3431.26, 3431.3, 3461.3, 3491.3, 3521.3, 3551.3, 3581.3, 3611.3, 3641.3, 3671.3, 3701.3, 3731.3, 3761.3, 3791.3, 3821.3, 

## Test 2 - advanced conversion

### Ontology

- `CurrentTimeCurve`
  - Predicates:
    - `isA -> Curve`
    - `isA -> TimeSeries`
    - `hasProperty -> Time`
    - `hasProperty -> Current`
    - Predicates:
      - `isA -> MeasuredProperty`
- `ChargeTimeCurve`?
  - Predicates:
    - `isA -> Curve`
    - `isA -> TimeSeries`
    - `hasProperty -> Time`
    - `hasProperty -> Charge`
    - Predicates:
      - `isA -> MeasuredProperty`
      - `isDerivedFrom -> Time`?
      - `isDerivedFrom -> Current`?

In [15]:
import numpy as np
from scipy.integrate import cumtrapz


def compute_capacities(t: np.ndarray, I: np.ndarray) -> np.ndarray:  # noqa: E741
    """Compute capacities as integral of current over time."""
    return cumtrapz(I, t, axis=0, initial=0)

In [16]:
from pathlib import Path

from tripper import EMMO, RDFS, Triplestore

import dlite

# Paths
thisdir = Path(".").absolute()
datadir = thisdir / "data"
modeldir = thisdir / "models"
dlite.storage_path.append(f"{modeldir}/*.json")

# Create collection -- our knowledge base in this example
coll = dlite.Collection()

# Create a triplestore "view" of our knowledge base
ts = Triplestore(backend="collection", collection=coll)

# Add namespaces
DON = ts.bind("don", "http://example.com/demo-ontology#")
CyclingTimeSeries = ts.bind(
    "CyclingTimeSeries",
    "https://example.com/meta/0.1/CyclingTimeSeries#",
)
ChargeTimeSeries = ts.bind(
    "ChargeTimeSeries",
    "https://example.com/meta/0.1/ChargeTimeSeries#",
)

# Load data
cycling_time_series = dlite.Instance.from_location(
    "CyclingStorageParser",
    f"{datadir}/test.json",
)
coll.add("CyclingTimeSeries", cycling_time_series)


# 1. Map input datamodels -- data provider + ontologist
# -----------------------------------------------------

# Ontologist adds missing concepts
# ts.add_triples([])

# Add mappings for the input data models -- data provider
ts.add_mapsTo(DON.Time, CyclingTimeSeries.t)
ts.add_mapsTo(DON.Current, CyclingTimeSeries.I)


# 2. Map output datamodels -- modeller + ontologist
# -------------------------------------------------

# Ontologist adds missing concepts
# ts.add_triples([])

# Add mappings for the output data model -- modeller
ts.add_mapsTo(DON.Time, ChargeTimeSeries.t)
ts.add_mapsTo(DON.Charge, ChargeTimeSeries.Q)


# 3. Add mappings for conversion functions -- ontologist
# ------------------------------------------------------
ts.add_function(
    compute_capacities,
    expects=[DON.Time, DON.Current],
    returns=[DON.Charge],
    standard="fno",
)


# 4. Instantiate a capacity vs. cycle data instance -- modeller
# -------------------------------------------------------------
(charge_time_series,) = coll.get_instances(
    metaid=ChargeTimeSeries,
    property_mappings=True,
    function_repo=ts.function_repo,
)

print("charge time series instance:")
print(charge_time_series)

charge time series instance:
{
  "007cbfd3-3436-4abf-9b10-f5d1077058fa":  {
    "meta": "https://example.com/meta/0.1/ChargeTimeSeries",
    "dimensions": {
      "N": 17384
    },
    "properties": {
      "t": [0, 0.0919598, 1.31488, 8.7852, 23.715, 30, 57.0177, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900, 930, 960, 990, 1020, 1050, 1080, 1110, 1140, 1170, 1200, 1230, 1260, 1290, 1320, 1350, 1380, 1410, 1440, 1470, 1500, 1530, 1560, 1590, 1620, 1650, 1680, 1710, 1740, 1770, 1800, 1830, 1860, 1890, 1920, 1950, 1980, 2010, 2040, 2070, 2100, 2130, 2160, 2190, 2220, 2250, 2280, 2310, 2340, 2370, 2400, 2430, 2460, 2490, 2520, 2550, 2580, 2610, 2640, 2670, 2700, 2730, 2760, 2790, 2820, 2850, 2880, 2910, 2940, 2970, 3000, 3030, 3060, 3090, 3120, 3150, 3180, 3210, 3240, 3270, 3300, 3330, 3360, 3390, 3420, 3431.26, 3431.3, 3461.3, 3491.3, 3521.3, 3551.3, 3581.3, 3611.3, 3641.3, 3671.3, 3701.3, 37

/opt/conda/lib/python3.9/site-packages/scipy/integrate/_quadrature.py:448: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  y = np.asarray(y)
/opt/conda/lib/python3.9/site-packages/scipy/integrate/_quadrature.py:452: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  x = np.asarray(x)
/opt/conda/lib/python3.9/site-packages/scipy/integrate/_quadrature.py:448: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  y = np.asarray(y)
/opt/conda/lib/python3.9/site-packages/scipy/integrate/_quadrature.py:452: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  x = np.asarray(x)
